In [64]:
import time
import numpy as np 
import pandas as pa 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import warnings
warnings.filterwarnings('ignore')

# Keras model building using SEQUENTIAL API
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import LSTM,CuDNNLSTM,CuDNNGRU,GRU,Dense,Flatten,Bidirectional,RepeatVector,TimeDistributed
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import EarlyStopping

In [7]:
## It contains all the models (LSTM,CuDNNLSTM,CuDNNGRU,GRU) 
## One to one and many to one
class MODELS_DEF:
    
    def __init__(self,X_data,Y_data):
        self.X = X_data
        self.Y = Y_data
        self.num_samples = len(self.X)
        self.timesteps = None
        self.tot_features = None

    def LSTM_MODEL(self,no_timestep,no_feature,early_stop = False):
        self.timesteps = no_timestep
        self.tot_features = no_feature
        if early_stop: 
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
        
        #Reshape the data (LSTM needs the data as a 3D shape (no_samples,timestep,feature))
        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        
        ##Sequential(keras) Model architecture
        
        model = Sequential()
        model.add(LSTM(64,activation='relu',input_shape=(no_timestep,no_feature)))
        model.add(Dense(1))
        model.compile(optimizer='adam',loss='mse')
        if early_stop:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0,callbacks=[es])
            end_time = time.time()
            total_time = start_time - end_time
        else:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0)
            end_time = time.time()
            total_time = start_time - end_time
        
        #Return the trained model 
        return model,total_time

    def CuDNNLSTM_MODEL(self,no_timestep,no_feature):
        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        model = Sequential()
        model.add(CuDNNLSTM(50,input_shape=(no_timestep,no_feature)))
        model.add(Dense(1))
        model.compile(optimizer='adam',loss='mse')
        start_time = time.time()
        model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0)
        end_time = time.time()
        total_time = start_time - end_time
        return model,total_time   
   
    def Bidirectional_LSTM(self,no_timestep,no_feature,early_stop = False):
        
        #Only use if time step more than 1
        if early_stop: 
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
        #Reshape the data (LSTM needs the data as a 3D shape (no_samples,timestep,feature))
        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        ##Sequential(keras) Model architecture
        model = Sequential()
        model.add(Bidirectional(LSTM(64,activation='relu',input_shape=(no_timestep,no_feature))))
        model.add(Dense(1))
        model.compile(optimizer='adam',loss='mse')
        if early_stop:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0,callbacks=[es])
            end_time = time.time()
            total_time = start_time - end_time
        else:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0)
            end_time = time.time()
            total_time = start_time - end_time
        #Return the trained model 
        return model,total_time
    

    def Stacked_CuDNNLSTM_MODEL(self,no_timestep,no_feature):

        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        model = Sequential()
        model.add(CuDNNLSTM(50,return_sequences=True,input_shape=(no_timestep,no_feature)))
        model.add(CuDNNLSTM(20))
        model.add(Dense(10,activation='relu'))  
        model.add(Dense(1))
        model.compile(optimizer='adam',loss='mse')
        start_time = time.time()
        model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0)
        end_time = time.time()
        total_time = start_time - end_time
        return model,total_time

    def Stacked_LSTM(self,no_timestep,no_feature,early_stop=False):
        if early_stop: 
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

        #Reshape the data (LSTM needs the data as a 3D shape (no_samples,timestep,feature))
        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        
        ##Sequential(keras) Model architecture
        model = Sequential()
        model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(no_timestep,no_feature)))
        model.add(LSTM(64,activation='relu'))
        model.add(Dense(10,activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam',loss='mse')
        
        if early_stop:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=3000,validation_split=0.1,batch_size=2,verbose=0,callbacks=[es])
            end_time = time.time()
            total_time = start_time - end_time
        else:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=3000,validation_split=0.1,batch_size=2,verbose=0)
            end_time = time.time()
            total_time = start_time - end_time
        #Return the trained model 
        return model,total_time

In [36]:
def model_train_pred(obj,pred_val,no_timestep,no_feature,Bidirectional=False):
    one_to_one_model,time_1 = obj.LSTM_MODEL(no_timestep,no_feature)
    one_to_one_model_es,time_2 = obj.LSTM_MODEL(no_timestep,no_feature,early_stop = True)
    one_to_one_cuda,time_3 = obj.CuDNNLSTM_MODEL(no_timestep,no_feature)
    one_to_one_cuda_st,time_4 = obj.Stacked_CuDNNLSTM_MODEL(no_timestep,no_feature)
    one_to_one_st,time_5 = obj.Stacked_LSTM(no_timestep,no_feature)
    if Bidirectional:
        one_to_one_bi,time_5 = obj.Bidirectional_LSTM(no_timestep,no_feature)
    
    ##Predict the value
    predict_reshape = np.reshape(predict_data,newshape=(1,obj.timesteps,obj.tot_features))
    
    print('Simple LSTM Withiout using early stopping {} TIME {}'.format(one_to_one_model.predict(predict_reshape),time_1))
    print('Simple LSTM With early stopping {} TIME {}'.format(one_to_one_model_es.predict(predict_reshape),time_2))
    print('CuDNNLSTM_MODEL {} TIME {}'.format(one_to_one_cuda.predict(predict_reshape),time_2))
    print('Stacked_CuDNNLSTM_MODEL {} TIME {}'.format(one_to_one_cuda_st.predict(predict_reshape),time_2))
    print('Stacked_LSTM {} TIME {}'.format(one_to_one_st.predict(predict_reshape),time_2))
    if Bidirectional:
        print('Bidirectional_LSTM {} TIME {}'.format(one_to_one_bi.predict(predict_reshape),time_2))

#### One to one sequence models with one features

In [4]:
## DATA OF ONE TO ONE SEQUENCE MODELS DATA (Timestep 1)
''' Suppose I have one data where I have only one feature with one time step and have only one target variable then it's a one to one sequence model '''

X_one = np.array([i for i in range(1,51)])
Y = np.array([x**2 for x in X_one])
print('FEATURE ONE \n {}'.format(X_one[:5]))
print('TARGET \n {}'.format(Y[:5]))

# Declare the timesteps and total features
no_timestep = 1 
no_feature = 1

predict_data = [60]
one_to_one = MODELS_DEF(X_one,Y)
model_train_pred(one_to_one,predict_data,no_timestep,no_feature)

plt.figure(figsize=(20,5))
sn.lineplot(X_one,Y)
plt.xlabel('Feature')
plt.ylabel('Target')

FEATURE ONE 
 [1 2 3 4 5]
TARGET 
 [ 1  4  9 16 25]
Epoch 00616: early stopping


#### One to one sequence models with more than one features


In [21]:
X_1 = np.array([i for i in range(1,30)])
X_2 = np.array([i for i in range(1,30)])

Y = X_1 + X_2

## For giving this in a sequctial api lstm we have to make a stacked feature
X = np.column_stack((X_1,X_2))
no_timestep = 1
no_feature = 2

predict_data = [52,62]
#Expected Value 114

one_to_one = MODELS_DEF(X,Y)
model_train_pred(one_to_one,predict_data,no_timestep,no_feature)

#### Many to one sequence problems  (Multiple timestep with one feature)

In [95]:
X_data = np.array([i+1 for i in range(90)]).astype(np.float32)
X_data_new = np.reshape(X_data,newshape=(len(X_data)//3,3,1))
Y_data = np.zeros((30,1))
for i in range(len(X_data_new)):
    Y_data[i] = (X_data_new[i][0] + X_data_new[i][1] + X_data_new[i][2])
Y_data = np.reshape(Y_data,newshape=(30))

no_timestep = 3
no_feature = 1

#Y_data = np.array([int(x) for x in Y_data])
predict_data = [100,200,300]
#Expected value=600

# Give the data to the model
one_to_one = MODELS_DEF(X_data,Y_data,no_timestep,no_feature)
model_train_pred(one_to_one,predict_data)

In [75]:
## It contains all the models (LSTM,CuDNNLSTM,CuDNNGRU,GRU) 
## One to many and Many to many
class MODELS_DEF_2:
    
    def __init__(self,X_data,Y_data):
        self.X = X_data
        self.Y = Y_data
        self.num_samples = len(self.X)
        self.timesteps = None
        self.tot_features = None

    def LSTM_MODEL(self,no_timestep,no_feature,early_stop = False,is_end_dec=False):
        self.timesteps = no_timestep
        self.tot_features = no_feature
        if early_stop: 
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
        
        #Reshape the data (LSTM needs the data as a 3D shape (no_samples,timestep,feature))
        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        
        ##Sequential(keras) Model architecture
        
        model = Sequential()
        model.add(LSTM(64,activation='relu',input_shape=(no_timestep,no_feature)))
        if is_end_dec:
            model.add(RepeatVector(no_timestep))
            model.add(LSTM(64,activation='relu',return_sequences=True))
            model.add(TimeDistributed(Dense(1)))
        else:
            model.add(Dense(2))
            
        model.compile(optimizer='adam',loss='mse')
        if early_stop:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0,callbacks=[es])
            end_time = time.time()
            total_time = start_time - end_time
        else:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0)
            end_time = time.time()
            total_time = start_time - end_time
        
        #Return the trained model 
        return model,total_time

    def CuDNNLSTM_MODEL(self,no_timestep,no_feature,is_end_dec=False):
        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        model = Sequential()
        model.add(CuDNNLSTM(50,input_shape=(no_timestep,no_feature)))
        if is_end_dec:
            model.add(RepeatVector(no_timestep))
            model.add(CuDNNLSTM(50,activation='relu',return_sequences=True))
            model.add(TimeDistributed(Dense(1)))
        else:
            model.add(Dense(2))
        model.compile(optimizer='adam',loss='mse')
        start_time = time.time()
        model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0)
        end_time = time.time()
        total_time = start_time - end_time
        return model,total_time   
   
    def Bidirectional_LSTM(self,no_timestep,no_feature,early_stop = False,is_end_dec=False):
        
        #Only use if time step more than 1
        if early_stop: 
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
        #Reshape the data (LSTM needs the data as a 3D shape (no_samples,timestep,feature))
        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        ##Sequential(keras) Model architecture
        model = Sequential()
        model.add(Bidirectional(LSTM(64,activation='relu',input_shape=(no_timestep,no_feature))))
        if is_end_dec:
            model.add(RepeatVector(no_timestep))
            model.add(Bidirectional(LSTM(50,activation='relu',return_sequences=True)))
            model.add(TimeDistributed(Dense(1)))
        else:
            model.add(Dense(2))
        model.compile(optimizer='adam',loss='mse')
        if early_stop:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0,callbacks=[es])
            end_time = time.time()
            total_time = start_time - end_time
        else:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0)
            end_time = time.time()
            total_time = start_time - end_time
        #Return the trained model 
        return model,total_time
    

    def Stacked_CuDNNLSTM_MODEL(self,no_timestep,no_feature):

        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        model = Sequential()
        model.add(CuDNNLSTM(50,return_sequences=True,input_shape=(no_timestep,no_feature)))
        model.add(CuDNNLSTM(20))
        model.add(Dense(10,activation='relu'))  
        model.add(Dense(2))
        model.compile(optimizer='adam',loss='mse')
        start_time = time.time()
        model.fit(X_Reshaped,self.Y,epochs=2000,validation_split=0.1,batch_size=5,verbose=0)
        end_time = time.time()
        total_time = start_time - end_time
        return model,total_time

    def Stacked_LSTM(self,no_timestep,no_feature,early_stop=False):
        if early_stop: 
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

        #Reshape the data (LSTM needs the data as a 3D shape (no_samples,timestep,feature))
        if no_timestep == 1:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples,no_timestep,no_feature))
        else:
            X_Reshaped = np.reshape(self.X,newshape=(self.num_samples//no_timestep,no_timestep,no_feature))
        
        ##Sequential(keras) Model architecture
        model = Sequential()
        model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(no_timestep,no_feature)))
        model.add(LSTM(64,activation='relu'))
        model.add(Dense(10,activation='relu'))
        model.add(Dense(2))
        model.compile(optimizer='adam',loss='mse')
        
        if early_stop:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=3000,validation_split=0.1,batch_size=2,verbose=0,callbacks=[es])
            end_time = time.time()
            total_time = start_time - end_time
        else:
            #Train the model 
            start_time = time.time()
            model.fit(X_Reshaped,self.Y,epochs=3000,validation_split=0.1,batch_size=2,verbose=0)
            end_time = time.time()
            total_time = start_time - end_time
        #Return the trained model 
        return model,total_time

In [40]:
##One to many sequnce porblems 
X = np.array([i for i in range(1,51)])
Y = np.array([[i-1,i+1] for i in X])

n_timestep = 1
no_features = 1

one_to_many = MODELS_DEF_2(X,Y)
predict_data = np.array([60])
#Expected Value 59,61

model_train_pred(one_to_many,predict_data,n_timestep,no_features)

Epoch 00030: early stopping
Simple LSTM Withiout using early stopping [[59.094616 61.267723]] TIME -99.33276438713074
Simple LSTM With early stopping [[61.149155 65.4885  ]] TIME -4.132920742034912
CuDNNLSTM_MODEL [[51.04557  52.966076]] TIME -4.132920742034912
Stacked_CuDNNLSTM_MODEL [[45.78521 47.96461]] TIME -4.132920742034912
Stacked_LSTM [[59.219757 61.44614 ]] TIME -4.132920742034912


In [72]:
##Many to many sequence problems with multiple features
X = np.array([x1 for x1 in range(2, 121, 2)])
X2 = np.array([x2 for x2 in range(8, 127, 2)])
Y = np.array([y for y in range(14, 133, 2)])
X_tot = np.column_stack((X,X2))
Y = np.array(Y).reshape(20, 3, 1)

n_timestep = 3
no_features = 2


Many_to_many = MODELS_DEF_2(X_tot,Y)
Many_to_many_model,time = Many_to_many.LSTM_MODEL(n_timestep,no_features,is_end_dec=True)

In [74]:
X_pred= np.array([140,142,144])
X2_pred =np.array([146,148,150])
predict_data = np.column_stack((X_pred,X2_pred))
predict_data_reshape = np.reshape(predict_data,newshape=(1,3,2))
Many_to_many_model.predict(predict_data_reshape)

array([[[153.08325],
        [156.02219],
        [158.47722]]], dtype=float32)